In [2]:
import nltk
from collections import Counter
from tabulate import tabulate
import numpy as np

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
with open ("hp_fa.txt", "r") as f:
    text = f.read()

In [5]:
! git clone https://github.com/kharazi/persian-stopwords.git
%cd persian-stopwords/

Cloning into 'persian-stopwords'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 51 (delta 7), reused 5 (delta 5), pack-reused 42
Unpacking objects: 100% (51/51), 34.96 KiB | 483.00 KiB/s, done.
/content/persian-stopwords


In [6]:
with open ("persian", "r") as f:
    fa_StopWords = f.read()
fa_StopWords = fa_StopWords.replace('\ufeff', '')
fa_StopWords = fa_StopWords.split("\n")
fa_punct = fa_StopWords[:18]

In [7]:
sentences = nltk.sent_tokenize(text)

In [ ]:
sentences

In [34]:
def generate_ngram(input_txt, n):

  def remove_punct(sentence):
    for i in sentence:
      #print("i :",i)
      #print("ss: ",ss)
      if i in fa_punct:
        sentence = sentence.replace(i, '')
    return sentence

  #split sentences
  sentences = nltk.sent_tokenize(input_txt)

  all_sentences_tok = []

  for sent in sentences:
    # preproces
    sent = sent.replace('\ufeff', '')
    sent = sent.replace('\xa0', ' ')
    sent = remove_punct(sent)

    # 
    new_sent = nltk.word_tokenize(sent)
    
    all_sentences_tok.append(new_sent)
    #print("new_sent :", all_sentences_tok)
  
  sentences_padn = [list(nltk.lm.preprocessing.pad_both_ends(sent,  2)) 
                        for sent in all_sentences_tok]

  if n > 2 :
    sentences_padn_new = []
    for sen in sentences_padn:
      s = ['<s>'] * (n-1)
      sentences_padn_new.append(s + sen)
    sentences_padn = sentences_padn_new

  
  sentences_ngram = [list(nltk.ngrams(sent, n)) 
                        for sent in sentences_padn]
  

  sentences_ngram_minus = [list(nltk.ngrams(sent, n-1)) 
                        for sent in sentences_padn]

  flat_ngram = sum(sentences_ngram, [])
  flat_ngram_minus = sum(sentences_ngram_minus, [])  

  return sentences_ngram, sentences_ngram_minus, flat_ngram, flat_ngram_minus           





def get_conditional_prob(ngrams_list, ngram_count, n_1_gram_count):
 
  probs = {}
  for bi in ngrams_list:

    cond = round((ngram_count[bi] + 1 ) / ( n_1_gram_count[bi[0]] + len(n_1_gram_count)), 6)
    
    probs[bi] = cond
    
  return(probs)



def find_max_prob(l, ngram_probs):

  b_tokens = {}
  max_l = []
  
  for k in ngram_probs :
    
    if k[0:len(l)] == tuple(l):
      
      b_tokens[k[len(l):][0]] = ngram_probs[k]

  sorted_list = sorted(b_tokens.items(), key=lambda item: item[1], reverse=True)
  
  for i in sorted_list[:4]:
    max_l.append(i[0])


  return max_l



In [39]:
def generate_sentence(text, n):

  sentences_ngram, sentences_n_1gram, flat_ngram, flat_n_1gram = generate_ngram(text, n)
  ngram_count = Counter(flat_ngram)
  n_1gram_count = Counter(flat_n_1gram)

  ngram_probs = get_conditional_prob(flat_ngram, ngram_count, n_1gram_count)
  
  gen_sentence = str()
  saw_letters = []
  nex_l_count = 0

  for i in range(24):

    if i < n-1 :
      letter = '<s>'
      gen_sentence += ' '
      gen_sentence += letter
      saw_letters.append(letter)
      nex_l_count +=1

    else:
    
      next_l = np.random.choice(find_max_prob(saw_letters[-(n-1):], ngram_probs))
      
      gen_sentence += ' '
      gen_sentence += next_l
      nex_l_count +=1
      saw_letters.append(next_l)
      letter= next_l

      if letter == '</s>' :
        break


  return gen_sentence, nex_l_count 


In [36]:
num_sen = 5
num = 0
n =2
while True:
  sen_gen, count_sen_gen = generate_sentence(text, n)
  if 12 <= count_sen_gen <=24 :
    print(sen_gen)
    num+=1
  if num == 5 :
    break

 <s> در همان وقت ها را روي زمين افتاد و از اين جا دور خود را از آن را به هري به او به
 <s> هاگريد با هم به نظر ميرسيد که در همان وقت به هري به او به او را از روي زمين کشيده شدن هري
 <s> در آن گاه در آن جا دور خود نشان داد و با حالتي عصبي و گفت من خودم شنيدم که از روي آن
 <s> رون به نظر ميرسيد هيچ وقت ها رو نگاه کرد که از اين جا </s>
 <s> هاگريد با صداي بلند گفت که در اين بار ديگر با اين کار را روي زمين کشيده و به او به هري گفت


In [37]:
num_sen = 5
num = 0
n = 3
while True:
  sen_gen, count_sen_gen = generate_sentence(text, n)
  if 12 <= count_sen_gen <=24 :
    print(sen_gen)
    num+=1
  if num == 5 :
    break


 <s> <s> <s> رون به او نزديک تر شدند هاگريد فرياد زد گريفندور </s>
 <s> <s> هري گفت خب که چي مي خواي نگذاري پسر ليلي و جيمز نه باورم نميشه که بالاخره طرف رفته باشه قرمز مي
 <s> <s> <s> <s> رون با عصبانيت از هري پاتر هري ميخواست با عجله به طرف در رفت وآمد بودند و به نظر نميرسيد
 <s> <s> هري به رون گفت اگه خودش دوست داشته باشه بذار ببينم دامبلدور فلوفي رو بزرگ کرده م بزرگ کردن اژدها ديگه برم
 <s> <s> در همان حال رداهاي جادوگريشان را درآوردند و کت يا ژاکت پوشيدند </s>


In [38]:
num_sen = 5
num = 0
n = 5
while True:
  sen_gen, count_sen_gen = generate_sentence(text, n)
  if 12 <= count_sen_gen <=24 :
    print(sen_gen)
    num+=1
  if num == 5 :
    break


 <s> <s> <s> <s> <s> <s> <s> هري به سرعت اوج ميگرفت و به سمت حلقه ي دروازه شيرجه ميرفت آن گاه از سوي
 <s> <s> <s> <s> در همان لحظه يک نفر از جلوي در قطار شروع به صحبت کرد اما سيوس فينيگان موحنايي به ميان حرف
 <s> <s> <s> <s> <s> رون گفت به نظر شما اگه بخوايم از اين جا رد بشيم بهمون حمله ميکنن هري گفت ممکنه ظاهراً
 <s> <s> <s> <s> رون به هري گفت همين طور بزن </s>
 <s> <s> <s> <s> <s> هري با اين که مطمئن نبود مار بتواند حرف او را بفهمد از لاي شيشه آهسته گفت مي دونم
